In [1]:
import pandas as pd
import numpy as np

from sklearn.utils import shuffle
import numpy as np

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout, Embedding, Masking, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend

In [2]:
df=pd.read_csv("C:\\Users\\ShreerangaKV\\Downloads\\NLP_ESA_QP_SEPT21-20220304T040459Z-001 (1)\\NLP_ESA_QP_SEPT21\\NLP1.csv")

In [3]:
df.head()

,Review,Star
0,This is my 1st ever Echo 🥺 I'm amazed by its c...,4.4
1,"Excellent product, super quality. It capable t...",5.0
2,So it is my first Echo Dot.It definitely looks...,2.0
3,I was expecting more as it's 4th gen... But di...,5.0
4,Love the new revamped version of Alexa. All y...,5.0


In [4]:
emotions=[]
for i in df.Star.iteritems():
    if i[1]>3:
         emotions.append("positive")
    else:
        emotions.append("negative")

In [5]:
df["Emotion"]=pd.Series(emotions)

In [6]:
df.Review=df.Review.str.lower()

In [7]:
df.Review=df.Review.str.replace('[^a-z\s]','')

C:\Users\SHREER~1\AppData\Local\Temp/ipykernel_22244/288346799.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df.Review=df.Review.str.replace('[^a-z\s]','')


In [14]:
import nltk
from nltk.corpus import stopwords

In [16]:
stopWords=set(stopwords.words('english'))

In [17]:
from gensim.parsing.preprocessing import remove_stopwords

In [18]:
docs=df.Review.apply(remove_stopwords)

In [19]:
docs[0]

'st echo im amazed capabilities feel necessities itthe want dependent dependentits impulsive purchase experience echo devices experience technology things useful like asking questions weather alarm news etcnow pros love design good weight sturdy built good bass driver highs tweeter adding unbelievable clarity sound easily pair alexa app priced moderately connections smart plug smart bulbs easily play apple music u hv subscription auto pairing light cuts excellent sound quality wize feet power adaptor cable line mm connecting external speakers htcons need continuous power need continuous wifi connectioncolors black white blue black gives contrast lighting ring black doesnt dirty easily gives distinct lookthats friends ask questions'

In [21]:
from gensim.parsing.porter import PorterStemmer

In [22]:
stem=PorterStemmer()

In [23]:
docs=stem.stem_documents(docs)

In [24]:
from nltk.stem.wordnet import WordNetLemmatizer

In [25]:
lemma=WordNetLemmatizer()

In [30]:
lemmDocs=[]
for lines in docs:
    lemmDocs.append(lemma.lemmatize(lines))
    

In [40]:
final_x=pd.Series(lemmDocs)

In [43]:
final_x[0]

'st echo im amaz capabl feel necess itth want depend dependentit impuls purchas experi echo devic experi technolog thing us like ask question weather alarm new etcnow pro love design good weight sturdi built good bass driver high tweeter ad unbeliev clariti sound easili pair alexa app price moder connect smart plug smart bulb easili plai appl music u hv subscript auto pair light cut excel sound qualiti wize feet power adaptor cabl line mm connect extern speaker htcon need continu power need continu wifi connectioncolor black white blue black give contrast light ring black doesnt dirti easili give distinct lookthat friend ask question'

In [34]:
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
vocabSize = 20000

In [35]:
tokenizer=Tokenizer(num_words=vocabSize,split=' ')

In [46]:
tokenizer.fit_on_texts(final_x.values)

In [47]:
X=tokenizer.texts_to_sequences(final_x.values)

In [51]:
 # padd sequences

In [53]:
X=pad_sequences(X)
Y=pd.get_dummies(df.Emotion)

In [60]:
X.shape

(344, 444)

In [54]:
from sklearn.model_selection import train_test_split
x_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2)

In [56]:
print(x_train.shape,X_test.shape)
print(Y_train.shape,Y_test.shape)

(275, 444) (69, 444)
(275, 2) (69, 2)


# Model building 

In [97]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Embedding,LSTM,SpatialDropout1D,Flatten

In [102]:
model=Sequential()
model.add(Embedding(20000,128,input_length=X.shape[1]))
# model.add(SpatialDropout1D(0.4))
model.add(LSTM(196,dropout=0.2,recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.add(Flatten())
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_9 (Embedding)     (None, 444, 128)          2560000   
                                                                 
 lstm_7 (LSTM)               (None, 196)               254800    
                                                                 
 dense_6 (Dense)             (None, 2)                 394       
                                                                 
 flatten_2 (Flatten)         (None, 2)                 0         
                                                                 
Total params: 2,815,194
Trainable params: 2,815,194
Non-trainable params: 0
_________________________________________________________________
None


In [103]:
batch_size=5
model.fit(x_train,Y_train,epochs=2,batch_size=32,verbose=2)

Epoch 1/2
9/9 - 21s - loss: 0.6506 - accuracy: 0.7091 - 21s/epoch - 2s/step
Epoch 2/2
9/9 - 18s - loss: 0.5803 - accuracy: 0.7091 - 18s/epoch - 2s/step


In [92]:
print(X_test.shape)
print(X_test.T.shape)

(69, 444)
(444, 69)


In [151]:
predictions=model.predict(X_test)

In [152]:
argMax=[]
for i in predictions:
    argMax.append(np.argmax(i))

In [153]:
type(argMax[0])

numpy.int64

In [160]:
Pred_Sentiment = []

In [161]:
for i in argMax:
    if i==1:
        Pred_Sentiment.append("Positive")
    else:
        Pred_Sentiment.append("Negative")

In [159]:
Pred_Sentiment

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positiv

In [162]:
af = Afinn()

NameError: name 'Afinn' is not defined

In [163]:
df.head()

,Review,Star,Emotion
0,this is my st ever echo im amazed by its capa...,4.4,positive
1,excellent product super quality it capable to ...,5.0,positive
2,so it is my first echo dotit definitely looks ...,2.0,negative
3,i was expecting more as its th gen but disappo...,5.0,positive
4,love the new revamped version of alexa all yo...,5.0,positive


In [164]:
Y=pd.get_dummies(df.Emotion)

In [165]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

In [170]:
x_train,x_test,y_train,y_test=train_test_split(final_x,Y,test_size=0.2)

In [171]:
countvec=CountVectorizer(min_df=2).fit(x_train)

In [172]:
train_dtm=countvec.transform(x_train)
test_dtm=countvec.transform(x_test)

In [173]:
x_train[0]

'st echo im amaz capabl feel necess itth want depend dependentit impuls purchas experi echo devic experi technolog thing us like ask question weather alarm new etcnow pro love design good weight sturdi built good bass driver high tweeter ad unbeliev clariti sound easili pair alexa app price moder connect smart plug smart bulb easili plai appl music u hv subscript auto pair light cut excel sound qualiti wize feet power adaptor cabl line mm connect extern speaker htcon need continu power need continu wifi connectioncolor black white blue black give contrast light ring black doesnt dirti easili give distinct lookthat friend ask question'

In [176]:
vocabSize = 6000

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=vocabSize, split=' ')
tokenizer.fit_on_texts(final_x.values)
X_train = tokenizer.texts_to_sequences(x_train.values)
X_train = pad_sequences(X_train)

X_test = tokenizer.texts_to_sequences(x_test.values)
X_test = pad_sequences(X_test)

In [177]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D

embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(vocabSize, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(lstm_out, dropout=0.1, recurrent_dropout=0.1))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_10 (Embedding)    (None, 444, 128)          768000    
                                                                 
 spatial_dropout1d_2 (Spatia  (None, 444, 128)         0         
 lDropout1D)                                                     
                                                                 
 lstm_8 (LSTM)               (None, 196)               254800    
                                                                 
 dense_7 (Dense)             (None, 2)                 394       
                                                                 
Total params: 1,023,194
Trainable params: 1,023,194
Non-trainable params: 0
_________________________________________________________________
None


In [178]:
model.fit(X_train, Y_train, epochs = 5, batch_size=batch_size, verbose = 2)

Epoch 1/5
55/55 - 78s - loss: 0.6578 - accuracy: 0.6945 - 78s/epoch - 1s/step
Epoch 2/5
55/55 - 108s - loss: 0.5400 - accuracy: 0.7164 - 108s/epoch - 2s/step
Epoch 3/5
55/55 - 112s - loss: 0.3479 - accuracy: 0.8618 - 112s/epoch - 2s/step
Epoch 4/5
55/55 - 26154s - loss: 0.1952 - accuracy: 0.9200 - 26154s/epoch - 476s/step
Epoch 5/5
55/55 - 89s - loss: 0.1659 - accuracy: 0.9236 - 89s/epoch - 2s/step
